In [53]:
%pip install psycopg2
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [54]:
import psycopg2
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Connect to DB

In [55]:
load_dotenv()
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    port=os.getenv("DB_PORT"),
)
cur = conn.cursor()

# Wifi Stats Data Preprocessing

In [56]:
query = "SELECT * FROM public.wifistats LIMIT 15"
cur.execute(query)
data = cur.fetchall()

In [57]:
wifi_df = pd.DataFrame(
    data,
    columns=[
        "label",
        "time",
        "ping_str",
        "signal (dBm)",
        "tx_bitrate_str",
        "rx_bitrate_str",
        "connected_time_str",
        "packet_loss",
    ],
)

wifi_df["connected_time (s)"] = (
    wifi_df["connected_time_str"].str.split(" ", expand=True)[0].astype(float)
)
wifi_df["tx_bitrate (MBit/s)"] = (
    wifi_df["tx_bitrate_str"].str.split(" ", expand=True)[0].astype(float)
)
wifi_df["rx_bitrate (MBit/s)"] = (
    wifi_df["rx_bitrate_str"].str.split(" ", expand=True)[0].astype(float)
)

ping_df = (
    wifi_df["ping_str"]
    .str.split("=", expand=True)[1]
    .str.replace(" ms", "", regex=False)
)
ping_df = ping_df.str.strip().str.split("/", expand=True).astype(float)
ping_df.columns = [
    "min_ping_rtt (ms)",
    "avg_ping_rtt (ms)",
    "max_ping_rtt (ms)",
    "mdev_ping_rtt (ms)",
]

wifi_df = pd.concat([wifi_df, ping_df], axis=1)
wifi_df = wifi_df.drop(
    ["connected_time_str", "tx_bitrate_str", "rx_bitrate_str", "ping_str"], axis=1
)
wifi_df

,label,time,signal (dBm),packet_loss,connected_time (s),tx_bitrate (MBit/s),rx_bitrate (MBit/s),min_ping_rtt (ms),avg_ping_rtt (ms),max_ping_rtt (ms),mdev_ping_rtt (ms)
0,raspi-e4:5f:01:ad:c9:11,2024-03-04 13:46:07.187989,-56.0,0.0,427480.0,65.0,26.0,3.813,5.140,9.264,2.091
1,raspi-e4:5f:01:88:c4:99,2024-03-04 13:46:07.188474,-46.0,0.0,427193.0,65.0,72.2,3.050,3.550,3.833,0.281
2,raspi-e4:5f:01:75:6a:df,2024-03-04 13:46:07.188907,-60.0,0.0,427880.0,162.0,200.0,3.337,4.234,5.007,0.569
3,raspi-e4:5f:01:6c:98:83,2024-03-04 13:46:07.189359,-52.0,0.0,427371.0,180.0,200.0,2.813,4.720,8.567,2.142
4,raspi-e4:5f:01:a0:4e:31,2024-03-04 13:46:07.189756,-58.0,0.0,379634.0,65.0,58.5,3.701,4.405,6.571,1.100
5,raspi-e4:5f:01:9c:25:7d,2024-03-04 13:46:07.190139,-50.0,0.0,174674.0,72.2,72.2,2.645,3.076,3.930,0.441
6,raspi-e4:5f:01:56:d9:a3,2024-03-04 13:46:07.190630,-66.0,0.0,427431.0,200.0,121.5,3.737,5.166,6.541,0.974
7,raspi-e4:5f:01:84:8c:99,2024-03-04 13:46:07.191025,-68.0,0.0,428520.0,200.0,180.0,3.960,4.415,4.655,0.243
8,raspi-e4:5f:01:a0:21:49,2024-03-04 13:46:07.191404,-46.0,0.0,427763.0,72.2,72.2,3.536,4.363,5.428,0.734
9,raspi-e4:5f:01:8d:07:57,2024-03-04 13:46:07.191790,-62.0,0.0,427614.0,72.2,43.3,2.955,4.156,7.327,1.600


# Device Data Preprocessing

In [58]:
query = "SELECT * FROM public.devices"
cur.execute(query)
data = cur.fetchall()

In [59]:
device_df = pd.DataFrame(
    data,
    columns=[
        "label",
        "ethernet_mac",
        "wlan_mac",
        "location",
        "tags",
        "comment",
        "uptime",
        "wireless_bytes",
        "wired_bytes",
        "last_seen",
    ],
)
display(device_df)

,label,ethernet_mac,wlan_mac,location,tags,comment,uptime,wireless_bytes,wired_bytes,last_seen
0,raspi-dc:a6:32:d7:6e:64,dc:a6:32:d7:6e:64,dc:a6:32:d7:6e:65,None,None,None,"18:11:40 up 206 days, 17:13, 0 users, load ...",7.723500e+07,1.246234e+10,2023-11-15 18:11:55.004674
1,raspi-e4:5f:01:72:89:99,e4:5f:01:72:89:99,e4:5f:01:72:89:9a,None,None,None,"17:43:43 up 154 days, 23:15, 0 users, load ...",5.499737e+07,7.350356e+09,2023-09-27 17:43:58.456155
2,raspi-e4:5f:01:72:a3:e5,e4:5f:01:72:a3:e5,e4:5f:01:72:a3:e6,None,None,None,None,NaN,NaN,NaT
3,raspi-e4:5f:01:56:d6:ce,e4:5f:01:56:d6:ce,e4:5f:01:56:d6:cf,None,None,None,"18:11:40 up 206 days, 17:06, 2 users, load ...",1.672108e+09,1.082823e+10,2023-11-15 18:11:55.017829
4,raspi-e4:5f:01:75:6e:53,e4:5f:01:75:6e:53,e4:5f:01:75:6e:54,None,None,None,"18:11:40 up 203 days, 23:40, 0 users, load ...",6.836665e+07,1.446518e+10,2023-11-15 18:11:55.003112
5,raspi-e4:5f:01:a0:4e:31,e4:5f:01:a0:4e:31,e4:5f:01:a0:4e:32,"Figueroa, 2507","[UCSB, Manzanita, Figueroa]",None,"22:47:15 up 128 days, 23:16, 0 users, load ...",2.026064e+09,0.000000e+00,2024-11-15 22:47:30.440915
6,raspi-e4:5f:01:a7:b1:c1,e4:5f:01:a7:b1:c1,e4:5f:01:a7:b1:c2,"Tepusquet, 2408","[UCSB, Manzanita, Tepusquet]",None,"22:47:15 up 387 days, 23:04, 2 users, load ...",1.423078e+11,0.000000e+00,2024-11-15 22:47:30.427161
7,raspi-e4:5f:01:8d:31:8b,e4:5f:01:8d:31:8b,e4:5f:01:8d:31:8e,SNL Lab Server Room,"[UCSB, Test, SNL Lab]",None,None,NaN,NaN,NaT
8,raspi-e4:5f:01:72:a2:eb,e4:5f:01:72:a2:eb,e4:5f:01:72:a2:f1,"Encino, 5119","[UCSB, Encino]",None,"00:03:04 up 106 days, 4:22, 4 users, load ...",2.739366e+09,9.649483e+12,2024-04-13 00:03:19.227382
9,raspi-e4:5f:01:a0:35:20,e4:5f:01:a0:35:20,e4:5f:01:a0:35:21,SNL Lab Server Room,"[UCSB, Test, SNL Lab]",None,None,NaN,NaN,NaT


In [60]:
cur.close()
conn.close()